# Image Denoising with Autoencoders

## Task 1: Introduction and Importing Libraries
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [ ]:
import numpy as np

from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.utils import to_categorical

%matplotlib inline

## Task 2: Data Preprocessing
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float')/255.0
x_test = x_test.astype('float')/255.0

In [ ]:
x_train = np.reshape(x_train, (60000, 784))
x_test = np.reshape(x_test,(10000, 784))

## Task 3: Adding Noise
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [ ]:
noise_factor = 0.9

x_train_noisy = x_train + np.random.rand(60000, 784)*noise_factor
x_train_noisy = np.clip(x_train_noisy, 0., 1.)

x_test_noisy = x_test + np.random.rand(10000, 784)*noise_factor
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
def plot(x, p, labels=False):
    plt.figure(figsize=(20,2))
    for i in range(10):
        plt.subplot(1,10,i+1)
        plt.imshow(x[i].reshape(28,28), cmap='binary')
        plt.xticks([])
        plt.yticks([])
        if labels:
            plt.xlabel(np.argmax(p[i]))
    plt.show()
plot(x_train, None)

In [ ]:
plot(x_train_noisy, None)

## Task 4: Building and Training a Classifier
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [ ]:
classifier = Sequential([
    Dense(256, activation='relu', input_shape=(784,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

classifier.compile(
    optimizer = 'adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

classifier.summary()

In [ ]:
history=classifier.fit(
    x_train, y_train,
    batch_size=512,
    epochs=5
)

In [ ]:
loss, accuracy = classifier.evaluate(x_test, y_test)
print(accuracy)

In [ ]:
loss, accuracy = classifier.evaluate(x_test_noisy, y_test)
print(accuracy)

## Task 5: Building the Autoencoder
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [ ]:
Input_image = Input(shape=(784,))

encoded = Dense(64, activation='relu')(Input_image)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(Input_image, decoded)
autoencoder.compile(
    optimizer = 'adam',
    loss ='binary_crossentropy',
    metrics=['accuracy']
)
autoencoder.summary()

## Task 6: Training the Autoencoder
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [ ]:
autoencoder.fit(
    x_train_noisy, x_train,
    validation_split=0.2,
    batch_size=512,
    epochs=100,
    verbose=False,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=5),
        LambdaCallback(on_epoch_end=lambda e,l: print('{:.3f}'.format(l['val_loss']), end=' _ '))
    ]
)

print(' _ ')
print('Training Done')

## Task 7: Denoised Images
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [ ]:
predictions = autoencoder.predict(x_test_noisy)

In [ ]:
plot(x_test_noisy, None)

In [ ]:
plot(predictions, None)

In [ ]:
loss, accuracy = classifier.evaluate(predictions, y_test)
print(accuracy)

## Task 8: Composite Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [ ]:
input_image = Input(shape=(784,))

x=autoencoder(input_image)
y=classifier(x)

denoise_and_classify = Model(input_image, y)

In [ ]:
predictions = denoise_and_classify.predict(x_test_noisy)

In [ ]:
plot(x_test_noisy, predictions, True)

In [ ]:
plot(x_test, to_categorical(y_test), True)